In [147]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import os

import matplotlib.pyplot as plt 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import VAR
from scipy.stats import pearsonr

pd.set_option('display.max_columns', 500)




In [148]:
files = os.listdir('ssw_events_saved_filtered/')

In [149]:
# Load all data into one dataframe
df_all = pd.DataFrame()
for file in files:
    df = pd.read_csv("ssw_events_saved_filtered/" + file)
    # add an t_ column that is 0 for when SSW_flah is true and increases by 1 after that value and decreases by one before that value
    df = df.sort_values(by='dates', ascending=True)
    df['central_date'] = df.iloc[21].dates
    df_all = pd.concat([df,df_all])
    
df_all = df_all.sort_values(by='dates', ascending=True)
df_all['index_'] = range(1, len(df_all) + 1)
# select year to a column
df_all['year'] = pd.DatetimeIndex(df_all['dates']).year

# load df_merged into a Pandas DataFrame and merge it with df_all
df_merged = pd.read_csv("df_merged.csv")

# merge df_all and df_merged on all common dates  
df_all = pd.merge(df_all, df_merged[['dates','mf_abs']], on='dates', how='inner')


In [150]:
df = df_all
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['t_delta'], y=df['wind_eastward_mean_cos'], name='wind_eastward_mean_selection', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['t_delta'], y=df['mf_abs'], name='mf_abs', line=dict(color='orange'), yaxis='y2'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-50, 80],
        title='wind_eastward_mean_cos'
    )
    ,yaxis2=dict(
        range=[0, 0.005],
        title='Temperature Selection Mean',
        side='right',
        overlaying='y'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude and Temperature, 1970 t0 2016',
    legend=dict(orientation="h", y=1.2, x=0.5))

# Show the plot
fig.show()

In [151]:
# group df by year and plot the mean of each year
df = df_all

df_grouped = df.groupby('central_date')

fig = go.Figure()

for year, dataframe in df_grouped:
    # plot the mean of each year using plotly
    # fig.add_trace(go.Scatter(x=df['t_delta'], y=df['wind_eastward_mean_cos'], name=f'ZMZW{year}', line=dict()))
    fig.add_trace(go.Scatter(x=dataframe['t_delta'], y=dataframe['mf_abs'], name=f'Momentum Flux ({year})', line=dict(), yaxis='y2'))

    
fig.update_layout(
    yaxis=dict(
        range=[-20, 50],
        title='ZMZW in m/s'
    )
    ,yaxis2=dict(
        range=[-0.0, 0.005],
        title='Absolute Momentum Flux',
        side='right',
        overlaying='y'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude and Temperature, 1970 t0 2016',
    legend=dict(orientation="h", y=1.2, x=0.5))

# Show the plot
fig.show()

In [152]:
avgs = df.mean()
devs = df.std()

for col in ['mf_abs', 'wind_eastward_mean_cos']:
    df[col] = (df[col] - avgs.loc[col]) / devs.loc[col]

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_98023/3266348371.py:1: FutureWarning:

The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_98023/3266348371.py:2: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



In [153]:
df

,dates,wind_eastward_mean,wind_eastward_mean_selection,temperature_mean_selection,wind_eastward_mean_cos,SSW_flag,t_delta,central_date,index_,year,mf_abs
0,2002-12-28,30.166703,15.994463,213.120404,1.463991,False,-20,2003-01-18,1,2002,0.055780
1,2002-12-29,24.246168,23.902853,236.805942,1.090984,False,-19,2003-01-18,2,2002,0.052392
2,2002-12-30,18.283184,27.995896,254.121855,0.715302,False,-18,2003-01-18,3,2002,-0.252742
3,2002-12-31,14.298763,27.821467,255.428372,0.464274,False,-17,2003-01-18,4,2002,-0.256181
4,2003-01-01,12.900051,39.659105,245.135665,0.376152,False,-16,2003-01-18,5,2003,-0.335193
...,...,...,...,...,...,...,...,...,...,...,...
393,2018-02-27,1.959706,-28.169621,240.048052,-0.313115,False,16,2018-02-12,406,2018,-0.602398
394,2018-02-28,7.124685,-21.560767,241.991318,0.012290,False,17,2018-02-12,407,2018,-0.545286
395,2018-03-01,8.236222,-10.779783,227.130560,0.082320,False,18,2018-02-12,408,2018,-0.586323
396,2018-03-02,8.793968,-4.203825,228.684507,0.117459,False,19,2018-02-12,409,2018,-0.601893


In [154]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['index_'], y=df['wind_eastward_mean_cos'], name='wind_eastward_mean_selection', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['index_'], y=df['mf_abs'], name='mf_abs', line=dict(color='orange'), yaxis='y2'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-5, 5],
        title='wind_eastward_mean_cos'
    )
    ,yaxis2=dict(
        range=[-5, 5],
        title='mf_abs',
        side='right',
        overlaying='y'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude and Temperature, 1970 t0 2016',
    legend=dict(orientation="h", y=1.2, x=0.5))

# Show the plot
fig.show()

In [155]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['index_'], y=df['wind_eastward_mean_cos'], name='wind_eastward_mean_selection', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['index_'], y=df['mf_abs'], name='mf_abs', line=dict(color='orange'), yaxis='y2'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-5, 5],
        title='wind_eastward_mean_cos'
    )
    ,yaxis2=dict(
        range=[-5, 5],
        title='mf_abs',
        side='right',
        overlaying='y'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude and Temperature, 1970 t0 2016',
    legend=dict(orientation="h", y=1.2, x=0.5))

# Show the plot
fig.show()

In [248]:
df_all

,dates,wind_eastward_mean,wind_eastward_mean_selection,temperature_mean_selection,wind_eastward_mean_cos,SSW_flag,t_delta,central_date,index_,year,mf_abs
0,2002-12-28,30.166703,15.994463,213.120404,1.463991,False,-20,2003-01-18,1,2002,0.055780
1,2002-12-29,24.246168,23.902853,236.805942,1.090984,False,-19,2003-01-18,2,2002,0.052392
2,2002-12-30,18.283184,27.995896,254.121855,0.715302,False,-18,2003-01-18,3,2002,-0.252742
3,2002-12-31,14.298763,27.821467,255.428372,0.464274,False,-17,2003-01-18,4,2002,-0.256181
4,2003-01-01,12.900051,39.659105,245.135665,0.376152,False,-16,2003-01-18,5,2003,-0.335193
...,...,...,...,...,...,...,...,...,...,...,...
393,2018-02-27,1.959706,-28.169621,240.048052,-0.313115,False,16,2018-02-12,406,2018,-0.602398
394,2018-02-28,7.124685,-21.560767,241.991318,0.012290,False,17,2018-02-12,407,2018,-0.545286
395,2018-03-01,8.236222,-10.779783,227.130560,0.082320,False,18,2018-02-12,408,2018,-0.586323
396,2018-03-02,8.793968,-4.203825,228.684507,0.117459,False,19,2018-02-12,409,2018,-0.601893


In [269]:
# group df by central_date
main_params_df = pd.DataFrame()

var = 'temperature_mean_selection'

df_grouped = df_all.groupby('central_date')

for central_date, dataframe in df_grouped:
    print('=============>' ,central_date)

    avgs = dataframe.mean(numeric_only=True)
    devs = dataframe.std(numeric_only=True)

    for col in [var, 'wind_eastward_mean_cos']:
        dataframe[col] = (dataframe[col] - avgs.loc[col]) / devs.loc[col]

    
    # plot_pacf(df['wind_eastward_mean_cos'])
    # plt.show()
    # plot_pacf(df['mf_abs'])
    # plt.show()


    for lag in range(1, 20):
        temp_series = dataframe[var].iloc[lag:]
        wind_series = dataframe['wind_eastward_mean_cos'].iloc[:-lag]
        # print('Lag: %s'%lag)
        # print(pearsonr(temp_series, wind_series))
        # print('------')

    dataframe = dataframe[['wind_eastward_mean_cos',var]]
    model = VAR(dataframe)
    model_fit = model.fit(maxlags=10)
    
    df_params = model_fit.params

    # add name to first column in df_params
    df_params =df_params.reset_index()

    # exclude first row from df_params
    df_params = df_params.iloc[1:]

    standard_error = model_fit.pvalues
    # rename column headers in standard_error
    standard_error = standard_error.reset_index()
    standard_error = standard_error.rename(columns={'wind_eastward_mean_cos':'wind_eastward_mean_cos_se',var:f'{var}_se'})
    standard_error = standard_error.iloc[1:]
    
    
    # create a new column with the first two characters of index
    df_params['var'] = df_params['index'].str[:3]

    # create a new column with the the characters after the first two characters of index
    df_params['lag'] = df_params['index'].str[3:]

    df_params['var'] = df_params['var'].apply(lambda x: x.strip('.'))
    df_params['lag'] = df_params['lag'].apply(lambda x: x.strip('.'))

    # select only columns lag where it's equal to wind_eastward_mean_cos
    df_params = df_params[df_params['lag'] == 'wind_eastward_mean_cos']

    df_params['identifier'] = central_date

    df_params = pd.merge(df_params, standard_error, on=['index'])

    # concatenate df_params to main_params_df
    main_params_df = pd.concat([main_params_df, df_params])



=============> 2003-01-18
=============> 2004-01-03
=============> 2006-01-23
=============> 2007-02-23
=============> 2008-02-23
=============> 2009-01-24
=============> 2010-02-05
=============> 2010-03-22
=============> 2013-01-08
=============> 2018-02-12


/Users/joaojesus/opt/anaconda3/envs/main_fyp_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/joaojesus/opt/anaconda3/envs/main_fyp_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/joaojesus/opt/anaconda3/envs/main_fyp_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/joaojesus/opt/anaconda3/envs/main_fyp_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/joaojesus/opt/anaconda3/envs/main_fyp_env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

An unsupported index was provided

In [270]:
# get probability of each coefficient
model_fit.pvalues



,wind_eastward_mean_cos,temperature_mean_selection
const,0.818245,0.022954
L1.wind_eastward_mean_cos,0.000001,0.202393
L1.temperature_mean_selection,0.395399,0.137631
L2.wind_eastward_mean_cos,0.034785,0.538978
L2.temperature_mean_selection,0.168683,0.130146
L3.wind_eastward_mean_cos,0.272515,0.777159
L3.temperature_mean_selection,0.267536,0.552886
L4.wind_eastward_mean_cos,0.510163,0.974183
L4.temperature_mean_selection,0.757818,0.743244
L5.wind_eastward_mean_cos,0.837009,0.752274


In [271]:
coefs_arr = []
for identifier, dataframe in main_params_df.groupby('identifier'):
    coefs_arr.append(dataframe['wind_eastward_mean_cos'].values)
    

In [272]:
coefs_arr_se = []
for identifier, dataframe in main_params_df.groupby('identifier'):
    coefs_arr_se.append(dataframe['wind_eastward_mean_cos_se'].values)

In [273]:
import plotly.express as px

dates = main_params_df['identifier'].unique()
dates.sort()

fig = px.imshow(coefs_arr, text_auto=True, aspect="auto")

# add title to the figure
# Add row labels to the figure
tickvals = list(enumerate(dates))

fig.update_layout(
    title="Correlation Matrix",
    yaxis=dict(
        tickmode='array', 
        tickvals=np.arange(len(dates)),
        ticktext=dates,
        title='Start Date of the SSW event'
    ),
    xaxis=dict(
        tickmode='array',
        tickvals=np.arange(np.shape(coefs_arr)[1]),
        title='Lag (in days)'
    )
)

fig.show()

In [274]:
import plotly.express as px

dates = main_params_df['identifier'].unique()
dates.sort()

fig = px.imshow(coefs_arr_se, text_auto=True, aspect="auto")

# add title to the figure
# Add row labels to the figure
tickvals = list(enumerate(dates))

fig.update_layout(
    title="Significance Matrix",
    yaxis=dict(
        tickmode='array', 
        tickvals=np.arange(len(dates)),
        ticktext=dates,
        title='Start Date of the SSW event'
    ),
    xaxis=dict(
        tickmode='array',
        tickvals=np.arange(np.shape(coefs_arr_se)[1]),
        title='Lag (in days)'
    ),
        coloraxis=dict(
        cmin=0,
        cmax=1
    )
)

fig.show()